In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ls
import os
os.chdir('/content/gdrive/My Drive/UROP')
!ls

gdrive	sample_data
1_Datapackage_preprocessing  datapackage  Upload_file.ipynb


# Datapackage处理

## 1.批量解压缩

- 需要将所有datapakage文件放在同一文件夹(datapakage文件夹下)
- 然后运行下面程序，会将所有文件解压缩到1_Datapackage_preprocessing文件夹下（程序会自行创建）
- 每个datapakage会被放在每个以时间戳为命名的文件夹下面

In [ ]:
#批量解压缩
"""
- 需要将所有datapakage文件放在同一文件夹(datapakage文件夹下)
- 然后运行下面程序，会将所有文件解压缩到1_Datapackage_preprocessing文件夹下（程序会自行创建）
- 每个datapakage会被放在每个以时间戳为命名的文件夹下面
"""

import os
import zipfile
import time
import datetime

datapackages_path = './datapackage/'
unzip_path = './1_Datapackage_preprocessing/'
if not os.path.isdir(datapackages_path):
  os.mkdir(datapackages_path)
if not os.path.isdir(unzip_path):
  os.mkdir(unzip_path)

datapackages = os.listdir(datapackages_path)
for datapackage in datapackages:
  print(datapackages_path + datapackage)
  zip_file = zipfile.ZipFile(datapackages_path + datapackage, "r")
  #存储datapakage的文件夹，以时间为名称
  datapackage_name = str(time.strftime("%Y%m%d_%H_%M_%S")) + str(datetime.datetime.now().microsecond)
  path = unzip_path+datapackage_name+'/'
  os.mkdir(path)
  #解压缩
  for fileName in zip_file.namelist():
    #略过自动生成的MACOSC文件夹
    if 'MACOSX' in fileName:
      continue
    zip_file.extract(fileName,path)


./datapackage/compathnion_data_package.zip


## 2.json文件解析


**遍历文件,根据不同文件向mongodb插入数据**
- SiteInfo.json
- Map.json
- Mapfile.zip
- Building.json
- OutdoorSite.json
- Floor.json
- Region.json
- LocSetting.json

**遍历过程中需要用到的功能**
1. 解析json中字段
2. 生成createTime和updateTime字段
3. 部分需要生成从json中拼接的新字段
4. 将zip转换成二进制数据字段
5. 将前面4点生成的字段拼接，并插入数据库

In [ ]:
"""
文件解析,根据不同文件向mongodb插入数据
解析逻辑：
1. 解析json中字段
2. 生成createTime和updateTime字段
3. 部分需要生成从json中拼接的新字段
4. 将zip转换成二进制数据字段
5. 将前面4点生成的字段拼接，并插入数据库
"""

#获取各类数据文件路径
root_path = r"./1_Datapackage_preprocessing"
siteinfo_json_list = []
building_json_list = []
outdoorsite_json_list = []
floor_json_list = []
region_json_list = []
locsetting_json_list = []
map_json_list = []

wifi_fingerprint_txt_list = []
ble_location_txt_list = []

for root,dirs,files in os.walk(root_path,topdown=True):
  for name in files:
    filepath = os.path.join(root,name)

    if 'SiteInfo.json' in filepath:
      siteinfo_json_list.append(filepath)
    elif 'Map.json' in filepath:
      map_json_list.append(filepath)
    elif 'Building.json' in filepath:
      building_json_list.append(filepath)
    elif 'Floor.json' in filepath:
      floor_json_list.append(filepath)
    elif 'Region.json' in filepath:
      region_json_list.append(filepath)
    elif 'OutdoorSite.json' in filepath:
      outdoorsite_json_list.append(filepath)
    elif 'LocSetting.json' in filepath:
      locsetting_json_list.append(filepath)
    elif 'WifiFingerprint.txt' in filepath:
      wifi_fingerprint_txt_list.append(filepath)
    elif 'BLELocation.txt' in filepath:
      ble_location_txt_list.append(filepath)

print(siteinfo_json_list)
print(building_json_list)
print(outdoorsite_json_list)
print(floor_json_list)
print(region_json_list)
print(locsetting_json_list)
print(map_json_list)
print(wifi_fingerprint_txt_list)
print(ble_location_txt_list)

['./1_Datapackage_preprocessing/20221007_13_47_12646890/compathnion_data_package/SiteInfo.json']
['./1_Datapackage_preprocessing/20221007_13_47_12646890/compathnion_data_package/Spatial Representation/Building.json']
[]
['./1_Datapackage_preprocessing/20221007_13_47_12646890/compathnion_data_package/Spatial Representation/00/Floor.json', './1_Datapackage_preprocessing/20221007_13_47_12646890/compathnion_data_package/Spatial Representation/01/Floor.json']
['./1_Datapackage_preprocessing/20221007_13_47_12646890/compathnion_data_package/Spatial Representation/00/00/Region.json', './1_Datapackage_preprocessing/20221007_13_47_12646890/compathnion_data_package/Spatial Representation/00/01/Region.json', './1_Datapackage_preprocessing/20221007_13_47_12646890/compathnion_data_package/Spatial Representation/00/02/Region.json', './1_Datapackage_preprocessing/20221007_13_47_12646890/compathnion_data_package/Spatial Representation/01/00/Region.json', './1_Datapackage_preprocessing/20221007_13_47_12

In [ ]:
###解析拼接json文件中各字段的函数，后续插入的时候调用
import time
import datetime
import json
import bson
import uuid
import math
from shapely.geometry import Polygon

def process_json(json_path):
  #解析json文件，返回字典
  with open (json_path,encoding="utf-8") as js:
    data = json.loads(js.read())
    return data
    
def get_time_attribute(type):
  #生成当前时间
  now_time = str(time.strftime('%Y-%m-%d %H:%M:%S'))
  if type == 'update':
    return {'updateTime':now_time}
  elif type == 'create':
    return {'createTime':now_time,'updateTime':now_time } 
  print("Please input 'type = update' or 'type = create' ")
  return 0

def insert_json(json_path,collection):
  """
  向某个collection插入某个json文件
  """
  #解析json文件
  json_data = process_json(json_path)
  #针对Floor.json和Region.json插入floor或regionID字段
  if 'FloorNo' in json_data.keys():
    FloorID = {'FloorID':json_data['ParentID'] + json_data['FloorNo']}
    json_data.update(FloorID)
  if 'RegionNo'in json_data.keys():
    RegionID = {'RegionID':json_data['ParentID'] + json_data['RegionNo']}
    json_data.update(RegionID)
  #针对map.json插入Data字段
  if( 'MapID' in json_data.keys() ) and ( 'MapFormat' in json_data.keys() ):
    mapfile_path = json_path[:-8]+'Mapfile.zip'
    binfile=open(mapfile_path,'rb')  #按二进制格式读取
    content = bson.binary.Binary(binfile.read())
    Data_attribute = {'Data':content}
    json_data.update(Data_attribute)
  #OutdoorSite fix
  if 'OutdoorSite.json' in json_path:
    SiteID = json_data.pop('OutdoorSiteID')
    json_data.update({'SiteID':SiteID})
    Name = json_data.pop('OutdoorSiteName')
    json_data.update({'Name':Name})

  #进行find检索
  if 'SiteInfo.json' in json_path:
    if json_data['IndoorSite'] == True:
      data = collection.find_one({'BuildingID':json_data['BuildingID']})
    else:
      data = collection.find_one({'OutdoorSiteID':json_data['OutdoorSiteID']})
  elif 'Map.json' in json_path:
    data = collection.find_one({'MapID':json_data['MapID']})
  elif 'Building.json' in json_path:
    data = collection.find_one({'BuildingID':json_data['BuildingID']})
  elif 'Floor.json' in json_path:
    data = collection.find_one({'FloorID':json_data['FloorID']})
  elif 'Region.json' in json_path:
    data = collection.find_one({'RegionID':json_data['RegionID']})
  elif 'OutdoorSite.json' in json_path:
    data = collection.find_one({'SiteID':json_data['SiteID']})
  elif 'LocSetting.json' in json_path:
    if 'BuildingID' in json_data.keys():
      data = collection.find_one({'BuildingID':json_data['BuildingID']})
    else:
      data = collection.find_one({'OutdoorSiteID':json_data['OutdoorSiteID']})

  #插入update/create time字段
  if data:
    create_time = {'createTime':data['createTime']}  #寻找createTime
    time_attribute = get_time_attribute('update')   #创造updateTime
    json_data.update(create_time)
    json_data.update(time_attribute)
    collection.delete_one(data)     #把之前的data覆盖掉
  else:
    time_attribute = get_time_attribute('create')
    json_data.update(time_attribute)
    
  #将组装好的json数据insert进mongodb的collection中
  collection.insert_one(json_data)
  return 0 

def process_txt(txt_path):
  #解析txt文件，返回字
  with open (txt_path) as f:
    data = f.readlines()
    return data

def insert_txt(txt_path,collection):
  """
  向某个collection插入某个txt文件
  """
  if 'WifiFingerprint.txt' in txt_path:
    #找buildingID
    json_path = txt_path[:-22]+'LocSetting.json'
    data = process_json(json_path)
    if 'BuildingID' in data.keys():
      buildingID = data['BuildingID']

    #解析txt文件
    txt_data = process_txt(txt_path)
  
    for i in txt_data:
      result = i.split("|")
        
      #组装json数据
      json_data = {}

      #抽出Latitude,Longitude,FloorID
      data = result[0].split(',')
      latitude = float(data[0])
      longitude = float(data[1])
      floorno = data[2]

      json_data.update({'RPID':str(uuid.uuid4())})
      json_data.update({'FloorID':buildingID+floorno})
      json_data.update({'Latitude':latitude})
      json_data.update({'Longitude':longitude})

      #抽出WIFIRssVector
      data = result[1].split(',')
      data[0] = data[0][1:]       #拿走[
      data[-1] = data[-1][:-2]     #拿走]\n
      wifirssvector = []
      for i in data:
        wifirssvector.append(i[1:-1])  #拿走""
                
      json_data.update({'WIFIRssVector':wifirssvector})

      #进行find检索
      data = collection.find_one({'Latitude':latitude, 'Longitude':longitude})

      #插入update/create time字段
      if data:
        json_data.update({'RPID':data['RPID']})
        create_time = {'createTime':data['createTime']}  #寻找createTime
        time_attribute = get_time_attribute('update')   #创造updateTime
        json_data.update(create_time)
        json_data.update(time_attribute)
        collection.delete_one(data)     #把之前的data覆盖掉
      else:
        time_attribute = get_time_attribute('create')
        json_data.update(time_attribute)

      #将组装好的json数据insert进mongodb的collection中
      collection.insert_one(json_data)

  elif 'BLELocation.txt' in txt_path:
    #找buildingID
    json_path = txt_path[:-18]+'LocSetting.json'
    data = process_json(json_path)
    if 'BuildingID' in data.keys():
      buildingID = data['BuildingID']
    else:
      OutdoorSiteID = data['OutdoorSiteID']

    #解析txt文件
    txt_data = process_txt(txt_path)

    for i in txt_data:
      result = i.split("|")

      #组装json数据
      json_data = {}

      #抽出data
      data = result[0].split(',')
      latitude = float(data[0])
      longitude = float(data[1])
      floorno = data[2]
      data = result[1].split(',')
      beacon_uuid = data[0]
      major = int(data[1])
      minor = int(data[2])

      json_data.update({'BeaconID':beacon_uuid+str(major)+str(minor)})
      if buildingID:
        json_data.update({'Indoor':True})
        json_data.update({'FloorID':buildingID+floorno})
      else:
        json_data.update({'Indoor':False})
        json_data.update({'SiteID':OutdoorSiteID})
      json_data.update({'Latitude':latitude})
      json_data.update({'Longitude':longitude})
      json_data.update({'UUID':beacon_uuid})
      json_data.update({'Major':major})
      json_data.update({'Minor':minor})

      #进行find检索
      data = collection.find_one({'BeaconID':json_data['BeaconID']})

      #插入update/create time字段
      if data:
        create_time = {'createTime':data['createTime']}  #寻找createTime
        time_attribute = get_time_attribute('update')   #创造updateTime
        json_data.update(create_time)
        json_data.update(time_attribute)
        collection.delete_one(data)     #把之前的data覆盖掉
      else:
        time_attribute = get_time_attribute('create')
        json_data.update(time_attribute)

      #将组装好的json数据insert进mongodb的collection中
      collection.insert_one(json_data)

  return 0

# return x-index, y-index givem lon lat (zoom-level: 20) ((lon, lat) -> (x,y))
def grid_lon_lat(lon,lat):
  x = math.floor((2**20)*(lon+180)/360)
  y = math.floor((1-math.log(math.tan(lat*math.pi/180)+1/math.cos(lat*math.pi/180))/math.pi)*2**19)
  return (x,y)
    
# print the 4 vertices of the grid given x-index, y-index (zoom-level: 20) (For checking)
def print_grid_lon_lat(x,y):
  print("("+str(360*x/(2**20)-180)+","+str(180/math.pi*(2*math.atan(math.exp((1-2*y/2**20)*math.pi))-math.pi/2))+")")
  print("("+str(360*x/(2**20)-180)+","+str(180/math.pi*(2*math.atan(math.exp((1-2*(y+1)/2**20)*math.pi))-math.pi/2))+")")
  print("("+str(360*(x+1)/(2**20)-180)+","+str(180/math.pi*(2*math.atan(math.exp((1-2*(y+1)/2**20)*math.pi))-math.pi/2))+")")
  print("("+str(360*(x+1)/(2**20)-180)+","+str(180/math.pi*(2*math.atan(math.exp((1-2*y/2**20)*math.pi))-math.pi/2))+")")
  return 0

# return all grids covered by the polygon given the boundary(vertices) (zoom-level: 20) (boundaries -> grids)
def grid_covered_by_boundary(boundary):
  boundary_xy = []
  grid = []
  valid_grid = []

  for i in boundary:
    x = (2**20)*(i[0]+180)/360
    y = (1-math.log(math.tan(i[1]*math.pi/180)+1/math.cos(i[1]*math.pi/180))/math.pi)*2**19
    boundary_xy.append((x,y))

  polygon = Polygon(boundary_xy)
  minx = math.floor(polygon.bounds[0])
  miny = math.floor(polygon.bounds[1])
  maxx = math.floor(polygon.bounds[2])
  maxy = math.floor(polygon.bounds[3])
    
  #create grids
  for lon in range(minx, maxx+1):
    for lat in range(miny, maxy+1):
      grid.append(Polygon([(lon,lat), (lon,lat+1), (lon+1,lat+1), (lon+1,lat)]))

  #validate if each grid intersects with polygon
  for i in grid:
    if polygon.intersects(i):
      valid_grid.append((int(i.bounds[0]),int(i.bounds[1])))

  return valid_grid

# initialize the GRID in database with processing the OUTDOORSITE
def outdoor_grid_init(db):
  data = db.OutdoorSite.find()
  for outdoorsite in data:
    boundary = []
    grid = []

    #retrieve the polygon
    for i in outdoorsite['Boundary']:
      lon = i[0]
      lat = i[1]
      boundary.append((lon,lat))
        
    #get grids that covered by the polygon
    grid = grid_covered_by_boundary(boundary)
    grididarray = []

    #put outdoorsiteID in grids (SiteID)
    for i in grid:
      json_data = {}
      zoom_level = 20
      x_index = i[0]
      y_index = i[1]
      json_data.update({'GridID':str(zoom_level*10**14+x_index*10**7+y_index)})
      json_data.update({'SiteID':outdoorsite['SiteID']})
      time_attribute = get_time_attribute('create')
      json_data.update(time_attribute)
      db.Grid.insert_one(json_data)

  return 0

# initialize the GRID in database with processing the FINGERPRINT
def indoor_grid_fingerprint_init(db):
  zoom_level = 20

  #for fingerprint (RPIDList)
  data = db.Fingerprint.find()
  for fingerprint in data:
    json_data = {}
    rpid_list = []

    rpid = fingerprint['RPID']
    floorID = fingerprint['FloorID']
    buildingID = floorID[:-2]
    lon = fingerprint['Longitude']
    lat = fingerprint['Latitude']
    (x,y) = grid_lon_lat(lon, lat)

    #put gridid in LocSetting if ShareSiteSignal==True
    gridID = str(zoom_level*10**14+x*10**7+y)+floorID
    find_loc = db.LocSetting.find_one({'BuildingID':buildingID})
    db.LocSetting.delete_one(find_loc)
    if find_loc['ShareSiteSignal'] == True:
      grididarray = []
      if 'SiteSignalGridID' in find_loc.keys():
        grididarray = find_loc['SiteSignalGridID']
      if gridID not in grididarray:
        grididarray.append(gridID)
      find_loc.update({'SiteSignalGridID':grididarray})
    db.LocSetting.insert_one(find_loc)

    gridID = {'GridID':gridID}
    json_data.update(gridID)
        
    #insert fingerprint into grid
    find_grid = db.Grid.find_one(gridID)
    if find_grid:
      if 'RPIDList' in find_grid.keys():
        rpid_list.extend(find_grid['RPIDList'])
      create_time = {'createTime':find_grid['createTime']}
      update_time = get_time_attribute('update')
      json_data.update(create_time)
      json_data.update(update_time)
      db.Grid.delete_one(find_grid)
    else:
      time_attribute = get_time_attribute('create')
      json_data.update(time_attribute)
    rpid_list.append(rpid)
    json_data.update({'RPIDList':rpid_list})

    db.Grid.insert_one(json_data)
  print('init fingerprint down')

# initialize the GRID in database with processing the BLELOCATION
def indoor_grid_blelocation_init(db):
  zoom_level = 20

  #for BLE (BeaconList)
  data = db.BLELocation.find()
  for blelocation in data:
    json_data = {}
    beacon_list = []

    beaconID = blelocation['BeaconID']
    indoor = blelocation['Indoor']
    lon = blelocation['Longitude']
    lat = blelocation['Latitude']
    (x,y) = grid_lon_lat(lon, lat)

    if indoor:
      floorID = blelocation['FloorID']
      gridID = str(zoom_level*10**14+x*10**7+y)+floorID
      buildingID = floorID[:-2]

      #put gridid in LocSetting if ShareSiteSignal==True
      find_loc = db.LocSetting.find_one({'BuildingID':buildingID})
      db.LocSetting.delete_one(find_loc)
      if find_loc['ShareSiteSignal'] == True:
        grididarray = []
        if 'SiteSignalGridID' in find_loc.keys():
          grididarray = find_loc['SiteSignalGridID']
        if gridID not in grididarray:
          grididarray.append(gridID)
        find_loc.update({'SiteSignalGridID':grididarray})
      db.LocSetting.insert_one(find_loc)
             
      gridID = {'GridID':gridID}
      json_data.update(gridID)
            
      find_grid = db.Grid.find_one(gridID)
    else:
      gridID = str(zoom_level*10**14+x*10**7+y)
      siteID = blelocation['SiteID']

      #put gridid in LocSetting if ShareSiteSignal==True
      find_loc = db.LocSetting.find_one({'OutdoorSiteID':siteID})
      db.LocSetting.delete_one(find_loc)
      if find_loc['ShareSiteSignal'] == True:
        grididarray = []
        if 'SiteSignalGridID' in find_loc.keys():
          grididarray = find_loc['SiteSignalGridID']
        if gridID not in grididarray:
          grididarray.append(gridID)
        find_loc.update({'SiteSignalGridID':grididarray})
      db.LocSetting.insert_one(find_loc)

      gridID = {'GridID':gridID}
      siteID = {'SiteID':siteID}
      json_data.update(gridID)
      json_data.update(siteID)

      find_grid = db.Grid.find_one({gridID, siteID})

    #insert iBeacon into grid
    if find_grid:
      if 'BeaconList' in find_grid.keys():
        beacon_list.extend(find_grid['BeaconList'])
      create_time = {'createTime':find_grid['createTime']}
      update_time = get_time_attribute('update')
      json_data.update(create_time)
      json_data.update(update_time)
      if 'RPIDList' in find_grid.keys():
        json_data.update({'RPIDList':find_grid['RPIDList']})
      db.Grid.delete_one(find_grid)
    else:
      time_attribute = get_time_attribute('create')
      json_data.update(time_attribute)

    beacon_list.append(beaconID)
    json_data.update({'BeaconList':beacon_list})

    db.Grid.insert_one(json_data)
  print('init BLELocation down')

# initialize the GRID in database with processing the REGION
def indoor_grid_region_init(db):
  zoom_level = 20

  #for connectedGrid (ConnectedGridID)
  data = db.Region.find()
  for region in data:
    transition_floorID = region['ParentID']

    for connectedregion in region['ConnectedRegions']:
      boundary = connectedregion['TransitionArea']
      transition_grid = grid_covered_by_boundary(boundary)

      for i in connectedregion['ArrivalArea']:
        arrival_grid = []

        boundary = i['Area']
        if i['RegionID'] == 'Outdoor':
          arrival_floorID = ''
        else:
          arrival_floorID = i['RegionID'][:-2]
        arrival_grid = grid_covered_by_boundary(boundary)

        for j in arrival_grid:
          arrival_gridID = str(20*10**14+j[0]*10**7+j[1])+arrival_floorID

          for grid in transition_grid:
            json_data = {}
            rpid_list = []
            beacon_list = []
            siteID = ""
            connectedgridID = []

            gridID = {'GridID':str(20*10**14+grid[0]*10**7+grid[1])+transition_floorID}

            #insert connected grids into grid
            found_grid = db.Grid.find_one(gridID)
            if found_grid:
              create_time = found_grid['createTime']
              update_time = get_time_attribute('update')

              json_data.update(gridID)
              if 'SiteID' in found_grid.keys():
                siteID.extend(found_grid['SiteID'])
                json_data.update({'SiteID':siteID})
              json_data.update({'createTime':create_time})
              json_data.update(update_time)
              if 'RPIDList' in found_grid.keys():
                rpid_list.extend(found_grid['RPIDList'])
                json_data.update({'RPIDList':rpid_list})
              if 'BeaconList' in found_grid.keys():
                beacon_list.extend(found_grid['BeaconList'])
                json_data.update({'BeaconList':beacon_list})
              if 'ConnectedGridID' in found_grid.keys():
                connectedgridID.extend(found_grid['ConnectedGridID'])
              if arrival_gridID not in connectedgridID:
                connectedgridID.append(arrival_gridID)
              json_data.update({'ConnectedGridID':connectedgridID})

              db.Grid.delete_one(found_grid)
                        
            else:
              connectedgridID.append(arrival_gridID)
              json_data.update(gridID)
              time_attribute = get_time_attribute('create')
              json_data.update(time_attribute)
              json_data.update({'ConnectedGridID':connectedgridID})

            db.Grid.insert_one(json_data)
  print('init region down')

# initialize the GRID in database
def indoor_grid_init(db):
  indoor_grid_fingerprint_init(db)
  indoor_grid_blelocation_init(db)
  indoor_grid_region_init(db)
  return 0

## 3.插入mongodb

In [ ]:
## 插入数据
import pymongo
from pymongo.database import Database
import pymongo

def mongodb_init():
  #连接
  mongo = pymongo.MongoClient(host='XX.XXX.XX.XXX',port=27017,username="XXXX",password="XXXX",authSource='XXXX')
  print('数据库当前的databases: ', mongo.list_database_names())

  return mongo

def get_db(mongo, db_name):
  db = Database(name = db_name, client=mongo)
  print('获取/创建库：', db.name)
  return db

#连接到mongo
mongo = mongodb_init()

#获取数据库对象
db = get_db(mongo,'loc_db')

#insert siteinfo
for json_file in siteinfo_json_list:
  insert_json(json_file, db.SiteInfo) 
#insert building
for json_file in building_json_list:
  insert_json(json_file, db.Building)
#insert outdoorsite
for json_file in outdoorsite_json_list:
  insert_json(json_file, db.OutdoorSite) 
#insert floor
for json_file in floor_json_list:
  insert_json(json_file, db.Floor) 
#insert region
for json_file in region_json_list:
  insert_json(json_file, db.Region) 
#insert locsetting
for json_file in locsetting_json_list:
  insert_json(json_file, db.LocSetting) 
#insert map
for json_file in map_json_list:
  insert_json(json_file, db.Map) 

#insert wififingerprint
for txt_file in wifi_fingerprint_txt_list:
  insert_txt(txt_file, db.Fingerprint)
#insert blelocation
for txt_file in ble_location_txt_list:
  insert_txt(txt_file, db.BLELocation)

#initialize collection GRID
outdoor_grid_init(db)
indoor_grid_init(db)

数据库当前的databases:  ['admin', 'config', 'loc_db', 'local', 'test', 'test1']
获取/创建库： loc_db
init fingerprint down
init BLELocation down
init region down


0

In [ ]:
## 插入数据
import pymongo
from pymongo.database import Database
import pymongo

def mongodb_init():
    #连接
    mongo = pymongo.MongoClient(host='XX.XXX.XX.XXX',port=27017,username="XXXX",password="XXXX",authSource='XXXX')

    print('数据库当前的databases: ', mongo.list_database_names())

    return mongo

def get_db(mongo, db_name):
    db = Database(name = db_name, client=mongo)
    print('获取/创建库：', db.name)
    return db

#连接到mongo
mongo = mongodb_init()

#获取数据库对象
db = get_db(mongo,'loc_db')

#insert outdoorsite
for json_file in outdoorsite_json_list:
  insert_json(json_file, db.OutdoorSite)


In [ ]:
## 插入数据
import pymongo
from pymongo.database import Database
import pymongo

def mongodb_init():
  #连接
  mongo = pymongo.MongoClient(host='XX.XXX.XX.XXX',port=27017,username="XXXX",password="XXXX",authSource='XXXX')
  print('数据库当前的databases: ', mongo.list_database_names())
  return mongo

def get_db(mongo, db_name):
  db = Database(name = db_name, client=mongo)
  print('获取/创建库：', db.name)
  return db

#连接到mongo
mongo = mongodb_init()

#获取数据库对象
db = get_db(mongo,'loc_db')

#db.SiteInfo.drop()
#db.Building.drop()
#db.OutdoorSite.drop()
#db.Floor.drop()
#db.Region.drop()
#db.LocSetting.drop()
#db.Map.drop()
#db.Fingerprint.drop()
#db.BLELocation.drop()
#db.Grid.drop()

#initialize collection GRID
#outdoor_grid_init(db)
#indoor_grid_init(db)

数据库当前的databases:  ['admin', 'config', 'loc_db', 'local', 'test', 'test1']
获取/创建库： loc_db
